In [9]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from conct_db import enginex
from datetime import date

**LEER LOS DATOS DE ORIGIEN //CARTERA DE REGISTROS//**

In [10]:
file = pd.read_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx', skipfooter=3, sheet_name='Export')
file1 = pd.DataFrame(file)
file1.to_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
# file.sample()

In [11]:
data1 = pd.read_csv('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.csv', encoding='UTF-8')
df = pd.DataFrame(data1)
# df.loc[[2134]]
# df.sample(3)
df.tail()

,Unnamed: 0,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
9270,9270,445625,TOTAL LITORAL S.A EN FORMACION,44562500987625,1,1.0,2021-12-18,1.0,1.0,2021-12-18,NO,1362.0
9271,9271,445625,TOTAL LITORAL S.A EN FORMACION,44562500987633,1,1.0,2021-10-28,1.0,1.0,2021-12-20,SI,1775.0
9272,9272,445625,TOTAL LITORAL S.A EN FORMACION,44562500987638,1,1.0,2021-11-19,1.0,1.0,2021-11-19,SI,2413.0
9273,9273,445625,TOTAL LITORAL S.A EN FORMACION,44562500987645,1,0.0,NaN,0.0,NaN,NaN,NO,NaN
9274,9274,445625,TOTAL LITORAL S.A EN FORMACION,44562500999111,1,1.0,2022-02-01,1.0,1.0,2022-02-01,SI,9199.0


In [12]:
df.columns

Index(['Unnamed: 0', 'Código Distribuidor', 'Distribuidor', 'POC', 'Clientes',
       'Registrado', 'Fecha registro', 'Adherible', 'Adherido',
       'Fecha adhesion', 'CanjeL3M', 'Puntos Disponibles'],
      dtype='object')

**TRANSFORMAR LOS DATOS PARA PODER CARGARLOS A LA BASE**

In [13]:
df['POC'] = df['POC'].astype(str)
# df['Fecha adhesion'] = df['Fecha adhesion'].convert_dtypes(int)
try:
    df.rename({' Fecha adhesion':'Fecha adhesion'}, axis=1, inplace=True)
except:
    pass
df['Fecha adhesion'] = pd.to_datetime(df['Fecha adhesion'],  format='%Y-%m-%d', errors='coerce')
df['Fecha registro'] = pd.to_datetime(df['Fecha registro'], format='%Y-%m-%d', errors='coerce')
# df['Fecha adhesion'] = df['Fecha adhesion'].dt.strftime('%m/%d/%Y')
df.tail()
# df.columns

,Unnamed: 0,Código Distribuidor,Distribuidor,POC,Clientes,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
9270,9270,445625,TOTAL LITORAL S.A EN FORMACION,44562500987625,1,1.0,2021-12-18,1.0,1.0,2021-12-18,NO,1362.0
9271,9271,445625,TOTAL LITORAL S.A EN FORMACION,44562500987633,1,1.0,2021-10-28,1.0,1.0,2021-12-20,SI,1775.0
9272,9272,445625,TOTAL LITORAL S.A EN FORMACION,44562500987638,1,1.0,2021-11-19,1.0,1.0,2021-11-19,SI,2413.0
9273,9273,445625,TOTAL LITORAL S.A EN FORMACION,44562500987645,1,0.0,NaT,0.0,NaN,NaT,NO,NaN
9274,9274,445625,TOTAL LITORAL S.A EN FORMACION,44562500999111,1,1.0,2022-02-01,1.0,1.0,2022-02-01,SI,9199.0


**METODO ALTERNATIVO PARA EXTRAER COD_DISTRI Y COD_CLIENTE**
# df['COD_CLIENTE'] = df.POC.str[-5:]
# df['COD_DISTRI'] = df.POC.str.extract('(\d{6,6}(?:\.\d+)?)')

In [14]:
df['COD_CLIENTE'] = df.POC.str.extract('(0{2,}\d+)')
df['COD_CLIENTE'] = df['COD_CLIENTE'].astype('Int64')
df.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
# df = df.fillna()
df[['Registrado', 'Adherible', 'Adherido']] = df[['Registrado', 'Adherible', 'Adherido']].astype('Float64')
df[['Registrado', 'Adherible', 'Adherido']] = df[['Registrado', 'Adherible', 'Adherido']].astype('Int64')
try:
    df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass
try:
    df.drop(columns=['POC', 'Distribuidor', 'Clientes'], inplace=True)
except:
    pass
df = df[['COD_DISTRI', 'COD_CLIENTE', 'Registrado', 'Fecha registro', 'Adherible', 'Adherido', 'Fecha adhesion', 'CanjeL3M', 'Puntos Disponibles']]
df.tail()

,COD_DISTRI,COD_CLIENTE,Registrado,Fecha registro,Adherible,Adherido,Fecha adhesion,CanjeL3M,Puntos Disponibles
9270,445625,987625,1,2021-12-18,1,1,2021-12-18,NO,1362.0
9271,445625,987633,1,2021-10-28,1,1,2021-12-20,SI,1775.0
9272,445625,987638,1,2021-11-19,1,1,2021-11-19,SI,2413.0
9273,445625,987645,0,NaT,0,<NA>,NaT,NO,NaN
9274,445625,999111,1,2022-02-01,1,1,2022-02-01,SI,9199.0


In [15]:

try:
    df.to_sql('BEES_Cartera_Registros', con=enginex, if_exists='replace', index=False)
    df.to_excel('z://PLANEAMIENTO/02-VENTAS/30-BEES/Cartera Registros.xlsx')
    print('Carga Exitosa')
except Exception as ex:
    print(ex)
    

(pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][SQL Server Native Client 11.0]TCP Provider: Se ha forzado la interrupción de una conexión existente por el host remoto.\r\n (10054) (SQLExecDirectW); [08S01] [Microsoft][SQL Server Native Client 11.0]Communication link failure (10054)')
[SQL: SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))]
[parameters: ('BASE TABLE', 'BEES_Cartera_Registros', 'dbo')]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
